In [38]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer
import random
import spacy
import boto3
import zipfile
import os
import json
from sense2vec import Sense2Vec
import requests
from collections import OrderedDict
import string
import pke
import nltk
import numpy 
from nltk import FreqDist
nltk.download('brown')
nltk.download('stopwords')
nltk.download('popular')
from nltk.corpus import stopwords
from nltk.corpus import brown
from similarity.normalized_levenshtein import NormalizedLevenshtein
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor
import time
from IPython.core.display import HTML

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\sanje\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sanje\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\sanje\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\sanje\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\sanje\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\sanje\AppData\Roaming\nltk

In [2]:
##This takes our list of potential outputs and the probability distribution already calculated — and chooses the option with the highest probability
def greedy_decoding (inp_ids,attn_mask,model,tokenizer):
  greedy_output = model.generate(input_ids=inp_ids, attention_mask=attn_mask, max_length=256)
  Question =  tokenizer.decode(greedy_output[0], skip_special_tokens=True,clean_up_tokenization_spaces=True)
  return Question.strip().capitalize()

##Returns a list of most likely output sequences
def beam_search_decoding (inp_ids,attn_mask,model,tokenizer):
  beam_output = model.generate(input_ids=inp_ids,
                                 attention_mask=attn_mask,
                                 max_length=256,
                               num_beams=10,
                               num_return_sequences=3,
                               no_repeat_ngram_size=2,
                               early_stopping=True
                               )
  Questions = [tokenizer.decode(out, skip_special_tokens=True, clean_up_tokenization_spaces=True) for out in
               beam_output]
  return [Question.strip().capitalize() for Question in Questions]

##Used for Story Generation
def topkp_decoding (inp_ids,attn_mask,model,tokenizer):
  topkp_output = model.generate(input_ids=inp_ids,
                                 attention_mask=attn_mask,
                                 max_length=256,
                               do_sample=True,
                               top_k=40,
                               top_p=0.80,
                               num_return_sequences=3,
                                no_repeat_ngram_size=2,
                                early_stopping=True
                               )
  Questions = [tokenizer.decode(out, skip_special_tokens=True,clean_up_tokenization_spaces=True) for out in topkp_output]
  return [Question.strip().capitalize() for Question in Questions]


In [3]:
##find the best-matching sense with the highest frequency
def MCQs_available(word,s2v):
    word = word.replace(" ", "_")
    sense = s2v.get_best_sense(word)
    if sense is not None:
        return True
    else:
        return False


def edits(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz '+string.punctuation
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)


##Removing near duplicates in semantic search retrieval
def sense2vec_get_words(word,s2v):
    output = []

    word_preprocessed =  word.translate(word.maketrans("","", string.punctuation))
    word_preprocessed = word_preprocessed.lower()

    word_edits = edits(word_preprocessed)

    word = word.replace(" ", "_")

    sense = s2v.get_best_sense(word)
    most_similar = s2v.most_similar(sense, n=15)

    compare_list = [word_preprocessed]
    for each_word in most_similar:
        append_word = each_word[0].split("|")[0].replace("_", " ")
        append_word = append_word.strip()
        append_word_processed = append_word.lower()
        append_word_processed = append_word_processed.translate(append_word_processed.maketrans("","", string.punctuation))
        if append_word_processed not in compare_list and word_preprocessed not in append_word_processed and append_word_processed not in word_edits:
            output.append(append_word.title())
            compare_list.append(append_word_processed)


    out = list(OrderedDict.fromkeys(output))

    return out


def get_options(answer,s2v):
    distractors =[]

    try:
        distractors = sense2vec_get_words(answer,s2v)
        if len(distractors) > 0:
            print(" Sense2vec_distractors successful for word : ", answer)
            return distractors,"sense2vec"
    except:
        print (" Sense2vec_distractors failed for word : ",answer)


    return distractors,"None"

##splitting a string, text into a list of tokens
def tokenize_sentences(text):
    sentences = [sent_tokenize(text)]
    sentences = [y for x in sentences for y in x]
    # Remove any short sentences less than 20 letters.
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

##Get the Sentences for the given keywords
def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        word = word.strip()
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values

    delete_keys = []
    for k in keyword_sentences.keys():
        if len(keyword_sentences[k]) == 0:
            delete_keys.append(k)
    for del_key in delete_keys:
        del keyword_sentences[del_key]

    return keyword_sentences

##Check score threshhold for word lists
def is_far(words_list,currentword,thresh,normalized_levenshtein):
    threshold = thresh
    score_list =[]
    for word in words_list:
        score_list.append(normalized_levenshtein.distance(word.lower(),currentword.lower()))
    if min(score_list)>=threshold:
        return True
    else:
        return False

#Filter the phrases    
def filter_phrases(phrase_keys,max,normalized_levenshtein ):
    filtered_phrases =[]
    if len(phrase_keys)>0:
        filtered_phrases.append(phrase_keys[0])
        for ph in phrase_keys[1:]:
            if is_far(filtered_phrases,ph,0.7,normalized_levenshtein ):
                filtered_phrases.append(ph)
            if len(filtered_phrases)>=max:
                break
    return filtered_phrases


#Build the Multipartite graph and rank candidates using random walk,
#Alpha controls the weight adjustment mechanism, see TopicRank for
#Threshold/method parameters.
def get_nouns_multipartite(text):
    out = []

    extractor = pke.unsupervised.MultipartiteRank()
    extractor.load_document(input=text, language='en')
    pos = {'PROPN', 'NOUN'}
    stoplist = list(string.punctuation)
    stoplist += stopwords.words('english')
    extractor.candidate_selection(pos=pos, stoplist=stoplist)
    
    try:
        extractor.candidate_weighting(alpha=1.1,
                                      threshold=0.75,
                                      method='average')
    except:
        return out

    keyphrases = extractor.get_n_best(n=10)

    for key in keyphrases:
        out.append(key[0])

    return out

##Get all the phrases from given texts
def get_phrases(doc):
    phrases={}
    for np in doc.noun_chunks:
        phrase =np.text
        len_phrase = len(phrase.split())
        if len_phrase > 1:
            if phrase not in phrases:
                phrases[phrase]=1
            else:
                phrases[phrase]=phrases[phrase]+1

    phrase_keys=list(phrases.keys())
    phrase_keys = sorted(phrase_keys, key= lambda x: len(x),reverse=True)
    phrase_keys=phrase_keys[:50]
    return phrase_keys


##Get keyworkds from given texts 
def get_keywords(nlp,text,max_keywords,s2v,fdist,normalized_levenshtein,no_of_sentences):
    doc = nlp(text)
    max_keywords = int(max_keywords)

    keywords = get_nouns_multipartite(text)
    keywords = sorted(keywords, key=lambda x: fdist[x])
    keywords = filter_phrases(keywords, max_keywords,normalized_levenshtein )

    phrase_keys = get_phrases(doc)
    filtered_phrases = filter_phrases(phrase_keys, max_keywords,normalized_levenshtein )

    total_phrases = keywords + filtered_phrases

    total_phrases_filtered = filter_phrases(total_phrases, min(max_keywords, 2*no_of_sentences),normalized_levenshtein )


    answers = []
    for answer in total_phrases_filtered:
        if answer not in answers and MCQs_available(answer,s2v):
            answers.append(answer)

    answers = answers[:max_keywords]
    return answers

##Generate MCQ's questions
def generate_questions_mcq(keyword_sent_mapping,device,tokenizer,model,sense2vec,normalized_levenshtein):
    batch_text = []
    answers = keyword_sent_mapping.keys()
    for answer in answers:
        txt = keyword_sent_mapping[answer]
        context = "context: " + txt
        text = context + " " + "answer: " + answer + " </s>"
        batch_text.append(text)

    encoding = tokenizer.batch_encode_plus(batch_text, pad_to_max_length=True, return_tensors="pt")


    print ("Running model for generation")
    input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

    with torch.no_grad():
        outs = model.generate(input_ids=input_ids,
                              attention_mask=attention_masks,
                              max_length=150)

    output_array ={}
    output_array["questions"] =[]
#     print(outs)
    for index, val in enumerate(answers):
        individual_question ={}
        out = outs[index, :]
        dec = tokenizer.decode(out, skip_special_tokens=True, clean_up_tokenization_spaces=True)

        Question = dec.replace("question:", "")
        Question = Question.strip()
        individual_question["question_statement"] = Question
        individual_question["question_type"] = "MCQ"
        individual_question["answer"] = val
        individual_question["id"] = index+1
        individual_question["options"], individual_question["options_algorithm"] = get_options(val, sense2vec)

        individual_question["options"] =  filter_phrases(individual_question["options"], 10,normalized_levenshtein)
        index = 3
        individual_question["extra_options"]= individual_question["options"][index:]
        individual_question["options"] = individual_question["options"][:index]
        individual_question["context"] = keyword_sent_mapping[val]
     
        if len(individual_question["options"])>0:
            output_array["questions"].append(individual_question)

    return output_array

#for normal one word questions
def generate_normal_questions(keyword_sent_mapping,device,tokenizer,model):  
    batch_text = []
    answers = keyword_sent_mapping.keys()
    for answer in answers:
        txt = keyword_sent_mapping[answer]
        context = "context: " + txt
        text = context + " " + "answer: " + answer + " </s>"
        batch_text.append(text)

    encoding = tokenizer.batch_encode_plus(batch_text, pad_to_max_length=True, return_tensors="pt")


    print ("Running model for generation")
    input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

    with torch.no_grad():
        outs = model.generate(input_ids=input_ids,
                              attention_mask=attention_masks,
                              max_length=150)

    output_array ={}
    output_array["questions"] =[]
    
    for index, val in enumerate(answers):
        individual_quest= {}
        out = outs[index, :]
        dec = tokenizer.decode(out, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        
        Question= dec.replace('question:', '')
        Question= Question.strip()

        individual_quest['Question']= Question
        individual_quest['Answer']= val
        individual_quest["id"] = index+1
        individual_quest["context"] = keyword_sent_mapping[val]
        
        output_array["questions"].append(individual_quest)
        
    return output_array

def random_choice():
    a = random.choice([0,1])
    return bool(a)


In [59]:
##Parth pretrained Model 
class QGen:
    
    def __init__(self):   
        self.tokenizer = T5Tokenizer.from_pretrained('t5-base')
        model = T5ForConditionalGeneration.from_pretrained('Parth/result')
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        # model.eval()
        self.device = device
        self.model = model
        self.nlp = spacy.load('en_core_web_sm')

        self.s2v = Sense2Vec().from_disk(r"C:\Users\sanje\Desktop\s2v_reddit_2015_md\s2v_old")

        self.fdist = FreqDist(brown.words())
        self.normalized_levenshtein = NormalizedLevenshtein()
        self.set_seed(42)
        
    def set_seed(self,seed):
        numpy.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)
            
    def predict_mcq(self, payload):
        start = time.time()
        inp = {
            "input_text": payload.get("input_text"),
            "max_questions": payload.get("max_questions", 4)
        }

        text = inp['input_text']
        sentences = tokenize_sentences(text)
        joiner = " "
        modified_text = joiner.join(sentences)


        keywords = get_keywords(self.nlp,modified_text,inp['max_questions'],self.s2v,self.fdist,self.normalized_levenshtein,len(sentences) )


        keyword_sentence_mapping = get_sentences_for_keyword(keywords, sentences)

        for k in keyword_sentence_mapping.keys():
            text_snippet = " ".join(keyword_sentence_mapping[k][:3])
            keyword_sentence_mapping[k] = text_snippet

   
        final_output = {}

        if len(keyword_sentence_mapping.keys()) == 0:
            return final_output
        else:
            try:
                generated_questions = generate_questions_mcq(keyword_sentence_mapping,self.device,self.tokenizer,self.model,self.s2v,self.normalized_levenshtein)

            except:
                return final_output
            end = time.time()

            final_output["statement"] = modified_text
            final_output["questions"] = generated_questions["questions"]
            final_output["time_taken"] = end-start
            
            if torch.device=='cuda':
                torch.cuda.empty_cache()
                
            return final_output


    ##Predict short questions with mapping of sentences and keywords
    def predict_shortq(self, payload):
        inp = {
            "input_text": payload.get("input_text"),
            "max_questions": payload.get("max_questions", 4)
        }

        text = inp['input_text']
        sentences = tokenize_sentences(text)
        joiner = " "
        modified_text = joiner.join(sentences)


        keywords = get_keywords(self.nlp,modified_text,inp['max_questions'],self.s2v,self.fdist,self.normalized_levenshtein,len(sentences) )


        keyword_sentence_mapping = get_sentences_for_keyword(keywords, sentences)
        
        for k in keyword_sentence_mapping.keys():
            text_snippet = " ".join(keyword_sentence_mapping[k][:3])
            keyword_sentence_mapping[k] = text_snippet

        final_output = {}

        if len(keyword_sentence_mapping.keys()) == 0:
            print('ZERO')
            return final_output
        else:
            
            generated_questions = generate_normal_questions(keyword_sentence_mapping,self.device,self.tokenizer,self.model)
            print(generated_questions)

            
        final_output["statement"] = modified_text
        final_output["questions"] = generated_questions["questions"]
        
        if torch.device=='cuda':
            torch.cuda.empty_cache()

        return final_output
            
    ##Predict paraphrases  
    def paraphrase(self,payload):
        start = time.time()
        inp = {
            "input_text": payload.get("input_text"),
            "max_questions": payload.get("max_questions", 3)
        }

        text = inp['input_text']
        num = inp['max_questions']
        
        self.sentence= text
        self.text= "paraphrase: " + self.sentence + " </s>"

        encoding = self.tokenizer.encode_plus(self.text,pad_to_max_length=True, return_tensors="pt")
        input_ids, attention_masks = encoding["input_ids"].to(self.device), encoding["attention_mask"].to(self.device)

        beam_outputs = self.model.generate(
            input_ids=input_ids,
            attention_mask=attention_masks,
            max_length= 50,
            num_beams=50,
            num_return_sequences=num,
            no_repeat_ngram_size=2,
            early_stopping=True
            )

#         print ("\nOriginal Question ::")
#         print (text)
#         print ("\n")
#         print ("Paraphrased Questions :: ")
        final_outputs =[]
        for beam_output in beam_outputs:
            sent = self.tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
            if sent.lower() != self.sentence.lower() and sent not in final_outputs:
                final_outputs.append(sent)
        
        output= {}
        output['Question']= text
        output['Count']= num
        output['Paraphrased Questions']= final_outputs
        
#         for i, final_output in enumerate(final_outputs):
#             print("{}: {}".format(i, final_output))

        if torch.device=='cuda':
            torch.cuda.empty_cache()
        
        return output




In [5]:
##t5-base model equence-to-sequence question generator which takes an answer and context as an input, and generates a question as an output
class BoolQGen:
       
    def __init__(self):
        self.tokenizer = T5Tokenizer.from_pretrained('t5-base')
        model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_boolean_questions')
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        # model.eval()
        self.device = device
        self.model = model
        self.set_seed(42)
        
    def set_seed(self,seed):
        numpy.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)

    def random_choice(self):
        a = random.choice([0,1])
        return bool(a)
    
    ##generate True/False Questions
    def predict_boolq(self,payload):
        start = time.time()
        inp = {
            "input_text": payload.get("input_text"),
            "max_questions": payload.get("max_questions", 4)
        }

        text = inp['input_text']
        num= inp['max_questions']
        sentences = tokenize_sentences(text)
        joiner = " "
        modified_text = joiner.join(sentences)
        answer = self.random_choice()
        form = "truefalse: %s passage: %s </s>" % (modified_text, answer)

        encoding = self.tokenizer.encode_plus(form, return_tensors="pt")
        input_ids, attention_masks = encoding["input_ids"].to(self.device), encoding["attention_mask"].to(self.device)

        output = beam_search_decoding (input_ids, attention_masks,self.model,self.tokenizer)
        if torch.device=='cuda':
            torch.cuda.empty_cache()
        
        final= {}
        final['Text']= text
        final['Count']= num
        final['Boolean Questions']= output
            
        return final
            


In [8]:
##t5-base model equence-to-sequence question generator which takes an answer and context as an input, and generates a question as an output
##Parth pretrained Model 
class AnswerPredictor:
          
    def __init__(self):
        self.tokenizer = T5Tokenizer.from_pretrained('t5-base')
        model = T5ForConditionalGeneration.from_pretrained('Parth/boolean')
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        # model.eval()
        self.device = device
        self.model = model
        self.set_seed(42)
        
    def set_seed(self,seed):
        numpy.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)

    def greedy_decoding (inp_ids,attn_mask,model,tokenizer):
        greedy_output = model.generate(input_ids=inp_ids, attention_mask=attn_mask, max_length=256)
        Question =  tokenizer.decode(greedy_output[0], skip_special_tokens=True,clean_up_tokenization_spaces=True)
        return Question.strip().capitalize()

    def predict_answer(self,payload):
        start = time.time()
        inp = {
            "input_text": payload.get("input_text"),
            "input_question" : payload.get("input_question")
        }

        context = inp["input_text"]
        question = inp["input_question"]
        input = "question: %s <s> context: %s </s>" % (question,context)

        encoding = self.tokenizer.encode_plus(input, return_tensors="pt")
        input_ids, attention_masks = encoding["input_ids"].to(self.device), encoding["attention_mask"].to(self.device)
        greedy_output = self.model.generate(input_ids=input_ids, attention_mask=attention_masks, max_length=256)
        Question =  self.tokenizer.decode(greedy_output[0], skip_special_tokens=True,clean_up_tokenization_spaces=True)
        output = Question.strip().capitalize()

        return output


In [9]:
import nltk
nltk.download('stopwords')
from pprint import pprint
qe= BoolQGen()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sanje\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:
##Read the CrisisLogger to dataframe
fields = ['upload_id', 'transcriptions']
logger_df = pd.read_csv(r'C:\Users\sanje\Desktop\ML-for-Good-Hackathon-main\ML-for-Good-Hackathon-main\Data\CrisisLogger\crisislogger.csv', usecols=fields)
print('Transcript: {}'.format(logger_df.transcriptions.iloc[24]))
payload = {
            "input_text": logger_df.transcriptions.iloc[24]
}

Transcript: I’m a woman, aged 63 (almost 64) and still working full time because of the change in the pension age. I was hospitalised with Swine flu in 2009 and it had a big effect on my health. I was diagnosed with asthma in 2010. My lungs have never fully recovered from the severe case of Swine flu. 
I normally work nine to five and there’s have been times over the past two years when I’ve felt very tired and weary. But it wasn’t until this pandemic hit that I actually felt old for the first time in my life. I’ll admit that I went into a blind spin panic when I realised how bad this pandemic was at the end of March. Stress being one of the triggers for my asthma, the feeling of dread and tightness in my chest caused my asthma to peak. I remembered how ill i was in 2009 and the feeling that I just wanted to die and due to my weakened lungs I couldn’t shake the fear. I rang my doctor (not wanting to visit the surgery) and she advised I double up my asthma medication and issued a prescr

In [62]:
##Generating Boolean Questions from above Transcripts
output = qe.predict_boolq(payload)
#pprint (output)

boolian_questions_df = pd.DataFrame(output.items(), columns=['Type', 'Data'])
boolian_df = pd.DataFrame(boolian_questions_df['Data'].iloc[2], columns=['BooleanQuestions']) 
display(HTML(boolian_df.to_html()))


,BooleanQuestions
0,Is there a cure for the swine flu?
1,Is there a cure for swine flu?
2,Is there such a thing as swine flu?


In [66]:
##Generating Paraphrased Questions from above Transcripts
qg = QGen()
output = qg.paraphrase(payload)
#pprint (output)

paraphrase_questions_df = pd.DataFrame(output.items(), columns=['Type', 'Data'])
paraphrase_df = pd.DataFrame(paraphrase_questions_df['Data'].iloc[2], columns=['ParaphrasedTarget']) 
display(HTML(paraphrase_df.to_html()))

,ParaphrasedTarget
0,"ParaphrasedTarget: I'm a woman, aged 63 (almost 64) and still working full time because of the change in the pension age. I was hospitalised with Swine flu in 2009 and it had"
1,"ParaphrasedTarget: I am a woman, aged 63 (almost 64) and still working full time because of the change in the pension age. I was hospitalised with Swine flu in 2009 and it had"
2,"ParaphrasedTarget: I'm a woman, 63 (almost 64) and still working full time because of the change in the pension age. I was hospitalised with Swine flu in 2009 and it had"


In [67]:
##Generate MCQ's and ther answers
output = qg.predict_mcq(payload)
#pprint (output)

mcq_questions_df = pd.DataFrame(output.items(), columns=['Type', 'Data'])
#display(HTML(faq_questions_df.to_html()))
##FAQ's DF
mcq_df = pd.DataFrame(mcq_questions_df['Data'].iloc[1]) 
display(HTML(mcq_df.to_html()))

Running model for generation
 Sense2vec_distractors successful for word :  asthma medication
 Sense2vec_distractors successful for word :  reflux
 Sense2vec_distractors successful for word :  supplies
 Sense2vec_distractors successful for word :  doctor


,question_statement,question_type,answer,id,options,options_algorithm,extra_options,context
0,What did my doctor tell me to double up on?,MCQ,asthma medication,1,"[Inhalers, Albuterol, Advair]",sense2vec,"[Sudafed, Own Medication, Nasal Spray, Antibiotics, Aspirin, Decongestants]",I rang my doctor (not wanting to visit the surgery) and she advised I double up my asthma medication and issued a prescription for extra supplies.
1,What was the cause of my wheezing?,MCQ,reflux,2,"[Constipation, Gerd, Stomach Issues]",sense2vec,"[Nausea, Heartburn, Dry Mouth, Ibs, Gi Problems]","Another call to my doctor, who said the reflux was caused by stress so I was given acid suppressants and signed off work until everything could settle down a bit. A few days later my wheezing had eased but I found I was suffering from severe reflux at nights."
2,What did my doctor tell me to double up my asthma medication and issue a prescription for?,MCQ,supplies,3,"[Rations, Building Materials, Stockpiles]",sense2vec,[],"I rang my doctor (not wanting to visit the surgery) and she advised I double up my asthma medication and issued a prescription for extra supplies. My son went to pick it up from the pharmacy the next day, only to discover they couldn’t get supplies of my usual medication."
3,Who advised me to double up my asthma medication and issued a prescription for extra supplies?,MCQ,doctor,4,"[Cardiologist, Doc, Primary Care Physician]",sense2vec,[General Practitioner],"Another call to my doctor, who said the reflux was caused by stress so I was given acid suppressants and signed off work until everything could settle down a bit. I rang my doctor (not wanting to visit the surgery) and she advised I double up my asthma medication and issued a prescription for extra supplies. Another call to my doctor and she prescribed an alternative to the medication I had been on since 2010 and my initial asthma diagnosis."


In [70]:
##Generate FAQ's and ther answers
output = qg.predict_shortq(payload)
#pprint (output)

faq_questions_df = pd.DataFrame(output.items(), columns=['Type', 'Data'])
#display(HTML(faq_questions_df.to_html()))
##FAQ's DF
faq_df = pd.DataFrame(faq_questions_df['Data'].iloc[1]) 
display(HTML(faq_df.to_html()))

Running model for generation
{'questions': [{'Question': 'What did my doctor tell me to double up on?', 'Answer': 'asthma medication', 'id': 1, 'context': 'I rang my doctor (not wanting to visit the surgery) and she advised I double up my asthma medication and issued a prescription for extra supplies.'}, {'Question': 'What was the cause of my wheezing?', 'Answer': 'reflux', 'id': 2, 'context': 'Another call to my doctor, who said the reflux was caused by stress so I was given acid suppressants and signed off work until everything could settle down a bit. A few days later my wheezing had eased but I found I was suffering from severe reflux at nights.'}, {'Question': 'What did my doctor tell me to double up my asthma medication and issue a prescription for?', 'Answer': 'supplies', 'id': 3, 'context': 'I rang my doctor (not wanting to visit the surgery) and she advised I double up my asthma medication and issued a prescription for extra supplies. My son went to pick it up from the pharmac

,Question,Answer,id,context
0,What did my doctor tell me to double up on?,asthma medication,1,I rang my doctor (not wanting to visit the surgery) and she advised I double up my asthma medication and issued a prescription for extra supplies.
1,What was the cause of my wheezing?,reflux,2,"Another call to my doctor, who said the reflux was caused by stress so I was given acid suppressants and signed off work until everything could settle down a bit. A few days later my wheezing had eased but I found I was suffering from severe reflux at nights."
2,What did my doctor tell me to double up my asthma medication and issue a prescription for?,supplies,3,"I rang my doctor (not wanting to visit the surgery) and she advised I double up my asthma medication and issued a prescription for extra supplies. My son went to pick it up from the pharmacy the next day, only to discover they couldn’t get supplies of my usual medication."
3,Who advised me to double up my asthma medication and issued a prescription for extra supplies?,doctor,4,"Another call to my doctor, who said the reflux was caused by stress so I was given acid suppressants and signed off work until everything could settle down a bit. I rang my doctor (not wanting to visit the surgery) and she advised I double up my asthma medication and issued a prescription for extra supplies. Another call to my doctor and she prescribed an alternative to the medication I had been on since 2010 and my initial asthma diagnosis."


In [73]:
#Question Answering - Summary (Simple)
answer = AnswerPredictor()
output = answer.predict_answer(payload)
pprint (output)


('I’m a woman, aged 63 (almost 64) and still working full time because of the '
 'change in the pension age. i was hospitalised with swine flu in 2009 and it '
 'had a big effect on my health. i was diagnosed with asthma in 2010. my lungs '
 'have never fully recovered from the severe case of swine flu. i normally '
 'work nine to five and there’s have been times over the past two years when '
 'i’ve felt very tired and weary. but it wasn’t until this pandemic hit')
